# Data Analysis
Import the libraries that will be used to try an explore the data

In [1]:
import numpy as np

#import pandad used for data frames to import and store the data set
import pandas as pd

#import re used for splitting up the time ordered into day month year
import re

#used to display pd dataframes
from IPython.display import display

## Import Data
Next import the provided .csv file and store it in a pandas dataframe object

In [2]:
data = pd.read_csv('data.csv')


Once the data is in a dataframe some simple analysis can be easily done. The mean, median and count of values can be easily calculated. 

In [3]:
#Print how many orders for each product
print(data["Product"].value_counts())


#Print how many orders from each Restruaunt
#print(data["Restaurant"].value_counts())

#Print how many order from restruaunt for each product is there for restruant 8
print(data["Product"][data["Restaurant"] == 8].value_counts())

#print Quantity min, max, mean
print(data[" Quantity"].min())
print(data[" Quantity"].max())
print(data[" Quantity"].mean())

38    913
Name: Product, dtype: int64
38    23
Name: Product, dtype: int64
1
8
1.43702081051


At the moment the column "Ordered At" is storing a string value as can be seen below.

In [4]:
type(data["Ordered At"][0])

str

This should be converted to a timestamp object as this will make the data more easy to use. 

In [5]:
def convert_timestamp(data):
    
    #Create new empty column in data set to store new timestamp object.
    if "Time Stamp" not in list(data.columns):
        data["Time Stamp"] = np.nan


    #Convert the string in the "Ordered At" column into a Pandas datestamp object and store
    # in new column

    for index, timestamp in data["Ordered At"].iteritems():
    
        #timestamp objects are in the form of US dates so convert the string into US date by switching the day and month
        timestampTemp = timestamp.split("/")
        timestampTemp1 = str(timestampTemp[1] + "/" + timestampTemp[0] + "/" + timestampTemp[2])
    
        #convert new string into timestamp object and store in new time stamp column
        data.set_value(index,"Time Stamp",pd.Timestamp(timestampTemp1))
    
    return data

In [7]:
data = convert_timestamp(data)
data.head()

,Restaurant,Product,Quantity,Price,Ordered At,Time Stamp
0,8,38,1,37.0,11/09/16 23:42,2016-09-11 23:42:00
1,161,38,1,54.5,12/09/16 15:37,2016-09-12 15:37:00
2,177,38,1,39.0,12/09/16 15:38,2016-09-12 15:38:00
3,28,38,1,40.0,12/09/16 15:53,2016-09-12 15:53:00
4,202,38,1,54.5,12/09/16 16:03,2016-09-12 16:03:00


As the times stamp is now stored as a timestamp object it is easier to pick out the month, day or year.

In [8]:
print(data["Time Stamp"][0].day)
print(data["Time Stamp"][0].month)
print(data["Time Stamp"][0].year)

11
9
2016


You can also then find out the amount of time between two timestamps

In [9]:
print((data["Time Stamp"][0] - data["Time Stamp"][200]))
print((data["Time Stamp"][0] - data["Time Stamp"][200]).days)
print((data["Time Stamp"][0] - data["Time Stamp"][200]).seconds)

-54 days +02:35:00
-54
9300


And also the min or max timestamp

In [10]:
print(data["Time Stamp"].min())
print(data["Time Stamp"].max())


2016-09-11 23:42:00
2017-02-06 17:37:00


In [ ]:
#print all orders by restaurant 8
display(data[(data["Restaurant"] == 8) & (data["Product"]==38)])
restaurant8 = data[(data["Restaurant"] == 8) & (data["Product"]==38)]

A function can be created which takes the dataset, restaurant number, & product number, and returns the data set but with an additional column stating when the next order of this product was placed by the same restaurant.

In [11]:
def add_next_order_date (data, restaurant, product):
    """
    Function that takes a data set and returns the same data set but with a new column giving the next order date next to 
    the product number given.
    
    """
    #If the next order date column exists do nothing, else create it and fill with blanks
    if "Next Order Date" not in list(data.columns):
        data["Next Order Date"] = np.nan
    
    #a temp data set is created to filter out the resturaunt orders of the requested preoduct
    tempData = data[(data["Restaurant"] == restaurant) & (data["Product"]==product)]
    
    #a for loop is used to loop through each of the items in the temp data set. The temp data set has the same index values
    #as in the original data set
    
    for index, timestamp in tempData["Time Stamp"].iteritems():
        
        # a blank list is used and then filled with every timestamp found to be larger than the one for the order
        timestamp_list = []
        next_order_found = False
        
        #another for loop is used to compare the time stamp of the order being considered to all other
        #orders of that product from that restaurant. This may not be very efficient and could maybe
        #be improved 
        for index2, timestamp2 in tempData["Time Stamp"].iteritems():
            
            #if the timestamp is greater i.e after the order timestamp add it to the list
            if timestamp2 > timestamp:
                timestamp_list.append(timestamp2)
                next_order_found = True
        #this if function means that the latest order of that product can be identified as
        #the column will still have the nan object in it.
        
        if next_order_found:
            #set the value of the next order date in the original data set to the minimum greater 
            #date found through the search
            data.set_value(index,"Next Order Date",min(timestamp_list))     

    return data

In [12]:
data_test = data

In [13]:
data_test[data_test["Restaurant"] == 161]

,Restaurant,Product,Quantity,Price,Ordered At,Time Stamp
1,161,38,1,54.5,12/09/16 15:37,2016-09-12 15:37:00
25,161,38,1,54.5,19/09/16 22:28,2016-09-19 22:28:00
63,161,38,1,54.5,28/09/16 16:20,2016-09-28 16:20:00
119,161,38,1,54.5,12/10/16 22:52,2016-10-12 22:52:00
152,161,38,1,54.5,26/10/16 14:54,2016-10-26 14:54:00
185,161,38,1,54.5,01/11/16 23:24,2016-11-01 23:24:00
255,161,38,1,54.5,16/11/16 16:06,2016-11-16 16:06:00


In [14]:
data_test = add_next_order_date(data_test,161,38)

In [15]:
data_test[data_test["Restaurant"]==161]

,Restaurant,Product,Quantity,Price,Ordered At,Time Stamp,Next Order Date
1,161,38,1,54.5,12/09/16 15:37,2016-09-12 15:37:00,2016-09-19 22:28:00
25,161,38,1,54.5,19/09/16 22:28,2016-09-19 22:28:00,2016-09-28 16:20:00
63,161,38,1,54.5,28/09/16 16:20,2016-09-28 16:20:00,2016-10-12 22:52:00
119,161,38,1,54.5,12/10/16 22:52,2016-10-12 22:52:00,2016-10-26 14:54:00
152,161,38,1,54.5,26/10/16 14:54,2016-10-26 14:54:00,2016-11-01 23:24:00
185,161,38,1,54.5,01/11/16 23:24,2016-11-01 23:24:00,2016-11-16 16:06:00
255,161,38,1,54.5,16/11/16 16:06,2016-11-16 16:06:00,NaN


This function can be modified slightly to instead provide the number of days between this order and the next order.

In [16]:
def add_next_order_days (data, restaurant, product):
    """
    Function that takes a data set and returns the same data set but with a new column giving the next order date next to 
    the product number given.
    
    """
    if "Next Order Days" not in list(data.columns):
        data["Next Order Days"] = np.nan
    
    tempData = data[(data["Restaurant"] == restaurant) & (data["Product"]==product)]
    
    for index, timestamp in tempData["Time Stamp"].iteritems():
        timestamp_list = []
        next_order_found = False
    
        for index2, timestamp2 in tempData["Time Stamp"].iteritems():
            if timestamp2 > timestamp:
                timestamp_list.append(timestamp2)
                next_order_found = True
    
        if next_order_found:
            next_date = min(timestamp_list)
            days = (next_date - timestamp).days
            data.set_value(index,"Next Order Days",days)     

    return data

In [18]:
data_test = add_next_order_days(data_test,161,38)
data_test[data_test["Restaurant"]==161]

,Restaurant,Product,Quantity,Price,Ordered At,Time Stamp,Next Order Date,Next Order Days
1,161,38,1,54.5,12/09/16 15:37,2016-09-12 15:37:00,2016-09-19 22:28:00,7.0
25,161,38,1,54.5,19/09/16 22:28,2016-09-19 22:28:00,2016-09-28 16:20:00,8.0
63,161,38,1,54.5,28/09/16 16:20,2016-09-28 16:20:00,2016-10-12 22:52:00,14.0
119,161,38,1,54.5,12/10/16 22:52,2016-10-12 22:52:00,2016-10-26 14:54:00,13.0
152,161,38,1,54.5,26/10/16 14:54,2016-10-26 14:54:00,2016-11-01 23:24:00,6.0
185,161,38,1,54.5,01/11/16 23:24,2016-11-01 23:24:00,2016-11-16 16:06:00,14.0
255,161,38,1,54.5,16/11/16 16:06,2016-11-16 16:06:00,NaN,NaN


In [19]:
# by creating a set of all the values in the restaurant column we can then loop through all the
# restaurants without duplication. 
resturaunt_set = set(data_test["Restaurant"])

for resturaunt in resturaunt_set:
    data_test = add_next_order_days (data_test,resturaunt,38)

In [29]:
data_test.head()

,Restaurant,Product,Quantity,Price,Ordered At,Time Stamp,Next Order Date,Next Order Days
0,8,38,1,37.0,11/09/16 23:42,2016-09-11 23:42:00,NaN,7.0
1,161,38,1,54.5,12/09/16 15:37,2016-09-12 15:37:00,2016-09-19 22:28:00,7.0
2,177,38,1,39.0,12/09/16 15:38,2016-09-12 15:38:00,NaN,23.0
3,28,38,1,40.0,12/09/16 15:53,2016-09-12 15:53:00,NaN,6.0
4,202,38,1,54.5,12/09/16 16:03,2016-09-12 16:03:00,NaN,NaN


We can create a function that calls all the functions we have previously created so that all the above data manipulation can be completed at the same time of importing the data.

In [21]:
def import_data(path):
    #import data to pandas data frame
    data = pd.read_csv(path)
    
    #creat list of all the restaurants used to loop functions.
    resturaunt_set = set(data_test["Restaurant"])
    
    #convert ordered at to timestamp
    data = convert_timestamp(data)
    
    #looping per resturaunt
    for resturaunt in resturaunt_set:
        data = add_next_order_days (data,resturaunt,38)
        data = add_next_order_date (data,resturaunt,38)
    
    return data

In [28]:
new_test_data = import_data("data.csv")
new_test_data.head()

,Restaurant,Product,Quantity,Price,Ordered At,Time Stamp,Next Order Days,Next Order Date
0,8,38,1,37.0,11/09/16 23:42,2016-09-11 23:42:00,7.0,2016-09-19 22:25:00
1,161,38,1,54.5,12/09/16 15:37,2016-09-12 15:37:00,7.0,2016-09-19 22:28:00
2,177,38,1,39.0,12/09/16 15:38,2016-09-12 15:38:00,23.0,2016-10-06 11:50:00
3,28,38,1,40.0,12/09/16 15:53,2016-09-12 15:53:00,6.0,2016-09-18 22:46:00
4,202,38,1,54.5,12/09/16 16:03,2016-09-12 16:03:00,NaN,NaN


## Data analysis and predictions
Create new function which sums up all the orders per restaurant and saves this info in a new data frame.

In [24]:
def resturaunt_orders_count(data, product):
    resturaunt_set = set(data["Restaurant"])
    orders_count = pd.DataFrame(columns = ("Restaurant","Total Orders"))

    for temp_restaurant in resturaunt_set:
        count = data["Product"][data["Restaurant"]==temp_restaurant].count()
        temp_frame = pd.DataFrame(data = [[temp_restaurant,count]],columns=["Restaurant","Total Orders"])
        orders_count = orders_count.append(temp_frame)

    orders_count = orders_count.reset_index(drop=True)
        
    return orders_count
    

In [25]:
test_rank = resturaunt_orders_count(new_test_data,38)

In [27]:
test_rank.sort_values("Total Orders", ascending=False).head()

,Restaurant,Total Orders
31,84.0,48.0
12,28.0,36.0
75,227.0,26.0
24,73.0,25.0
63,174.0,24.0


## Prediction Function

2No. functions which aim to predict the date the restaurant will make their next order based on how long it has been between orders previously. 

In [30]:
def dto_mean_predict(data,restaurant,product):
    """
    
    """
    temp_data = data[:][(data["Restaurant"]==restaurant)&(data["Product"]==product)]
    days_predict = temp_data["Next Order Days"].mean()
    last_order = temp_data["Time Stamp"][temp_data["Next Order Date"].isnull()].all()
    print(type(last_order))
    order_date_predict = last_order + pd.Timedelta(days_predict,'d')
    return order_date_predict

def dto_median_predict(data,restaurant,product):
    """
    
    """
    temp_data = data[:][(data["Restaurant"]==restaurant)&(data["Product"]==product)]
    days_predict = temp_data["Next Order Days"].median()
    last_order = temp_data["Time Stamp"][temp_data["Next Order Date"].isnull()].all()
    print(type(last_order))
    order_date_predict = last_order + pd.Timedelta(days_predict,'d')
    return order_date_predict

In [34]:
display(new_test_data[:][new_test_data["Restaurant"] == 84].tail())
dto_mean_predict(new_test_data,84,38)

,Restaurant,Product,Quantity,Price,Ordered At,Time Stamp,Next Order Days,Next Order Date
776,84,38,1,45.0,22/01/17 22:04,2017-01-22 22:04:00,3.0,2017-01-25 23:54:00
802,84,38,1,45.0,25/01/17 23:54,2017-01-25 23:54:00,4.0,2017-01-29 23:56:00
834,84,38,1,45.0,29/01/17 23:56,2017-01-29 23:56:00,1.0,2017-01-31 23:48:00
846,84,38,1,45.0,31/01/17 23:48,2017-01-31 23:48:00,3.0,2017-02-04 01:08:00
893,84,38,1,45.0,04/02/17 01:08,2017-02-04 01:08:00,NaN,NaN


<class 'pandas.tslib.Timestamp'>


Timestamp('2017-02-06 14:24:35.702400')

In [35]:
dto_median_predict(new_test_data,84,38)

<class 'pandas.tslib.Timestamp'>


Timestamp('2017-02-06 01:08:00')

Based on the prediction above and data currently loaded into new_test_data the next order for this product by restaurant 84 should be on 06/02/2017. 

In [ ]:
other_data = pd.read_csv('data1.csv')
other_data.tail()